# Analyzing Health and Nutrition Data

We'll start by importing relevant _libraries_ (i.e. prebuilt chunks of Python that have useful functions).

In [7]:
import pandas as pd    #data manipulation
import numpy as np     #mathematical operations

import matplotlib.pyplot as plt      #plotting tools
import geopandas as gpd              #geo mapping tools
import contextily as ctx             #map illustrating tool

from urllib.request import urlopen   #web url reading
import json                          #json reader
import xport                   #xport reader (us gov't data export format)

## Exploratory Data Analysis

### Demographics

We'll be importing data from the [National Health and Nutrition Examination Survey](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&CycleBeginYear=2017) as a pandas dataframe.  We can start by looking at the underlying demographics of the survey participants.

In [15]:
# download_url = "https://raw.githubusercontent.com/annahaensch/DataAndSocialJustice/main/Data/Health_and_Nutrition/DEMO_I.XPT"

url = "../Data/Health_and_Nutrition/DEMO_I.XPT"

with open(url, 'rb') as f:
    df = xport.to_dataframe(f)

In [16]:
df.head()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
0,93703.0,10.0,2.0,2.0,2.0,NaN,5.0,6.0,2.0,27.0,...,3.0,1.0,3.0,9246.491865,8539.731348,2.0,145.0,15.0,15.0,5.00
1,93704.0,10.0,2.0,1.0,2.0,NaN,3.0,3.0,1.0,33.0,...,3.0,1.0,2.0,37338.768343,42566.614750,1.0,143.0,15.0,15.0,5.00
2,93705.0,10.0,2.0,2.0,66.0,NaN,4.0,4.0,2.0,NaN,...,1.0,2.0,NaN,8614.571172,8338.419786,2.0,145.0,3.0,3.0,0.82
3,93706.0,10.0,2.0,1.0,18.0,NaN,5.0,6.0,2.0,222.0,...,3.0,1.0,2.0,8548.632619,8723.439814,2.0,134.0,NaN,NaN,NaN
4,93707.0,10.0,2.0,1.0,13.0,NaN,5.0,7.0,2.0,158.0,...,2.0,1.0,3.0,6769.344567,7064.609730,1.0,138.0,10.0,10.0,1.88


To understand the column headings here, we should consult the [NHANES documentation brochure](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm).  Who is missing from this data and why?  How should that impact our analysis?  We can look at the summary statistics of the dataframe with:

In [20]:
df.describe()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA,INDHHIN2,INDFMIN2,INDFMPIR
count,9254.000000,9254.0,9254.000000,9254.000000,9254.000000,597.000000,9254.000000,9254.000000,8704.000000,3433.000000,...,8764.000000,9063.000000,4751.000000,9254.000000,9254.000000,9254.000000,9254.000000,8763.000000,8780.000000,8023.000000
mean,98329.500000,10.0,1.940566,1.507564,34.334234,10.437186,3.233953,3.497190,1.517348,107.475677,...,2.050776,1.472691,2.110714,34670.706829,34670.706829,1.517614,140.965853,12.500057,12.202506,2.375490
std,2671.544029,0.0,0.236448,0.499970,25.500280,7.092970,1.277650,1.700961,0.499728,70.618237,...,0.652806,0.721168,0.688517,41356.667327,43343.996803,0.499717,4.200801,17.307571,17.155294,1.600291
min,93703.000000,10.0,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,2571.068712,0.000000,1.000000,134.000000,1.000000,1.000000,0.000000
25%,96016.250000,10.0,2.000000,1.000000,11.000000,4.000000,3.000000,3.000000,1.000000,43.000000,...,2.000000,1.000000,2.000000,13074.433246,12347.311890,1.000000,137.000000,6.000000,6.000000,1.040000
50%,98329.500000,10.0,2.000000,2.000000,31.000000,10.000000,3.000000,3.000000,2.000000,106.000000,...,2.000000,1.000000,2.000000,21098.454260,21059.894454,2.000000,141.000000,8.000000,8.000000,1.920000
75%,100642.750000,10.0,2.000000,2.000000,58.000000,17.000000,4.000000,4.000000,2.000000,166.000000,...,2.000000,2.000000,3.000000,36923.316352,37561.998020,2.000000,145.000000,14.000000,14.000000,3.690000
max,102956.000000,10.0,2.000000,2.000000,80.000000,24.000000,5.000000,7.000000,2.000000,239.000000,...,3.000000,3.000000,3.000000,433085.005262,419762.836488,2.000000,148.000000,99.000000,99.000000,5.000000


For any individual column we can get a closer look at the value counts with:

In [22]:
df["RIAGENDR"].value_counts()

2.0    4697
1.0    4557
Name: RIAGENDR, dtype: int64

### Dietary Data

We'll be importing dietary survey data from the [National Health and Nutrition Examination Survey](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&CycleBeginYear=2017) as a pandas dataframe.  To understand the data we can look in the accompanying [documentation brochure](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DR2IFF_I.htm).

In [23]:
url = "../Data/Health_and_Nutrition/DR2IFF_I.XPT"

with open(url, 'rb') as f:
    df_diet = xport.to_dataframe(f)

In [25]:
df_diet

,SEQN,WTDRD1,WTDR2D,DR2ILINE,DR2DRSTZ,DR2EXMER,DRABF,DRDINT,DR2DBIH,DR2DAY,...,DR2IM181,DR2IM201,DR2IM221,DR2IP182,DR2IP183,DR2IP184,DR2IP204,DR2IP205,DR2IP225,DR2IP226
0,83732.0,92670.699919,69945.934107,1.0,1.0,87.0,2.0,2.0,2.0,4.0,...,0.000,0.000,0.000,0.003,0.000,0.0,0.000,0.000,0.000,0.000
1,83732.0,92670.699919,69945.934107,2.0,1.0,87.0,2.0,2.0,2.0,4.0,...,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000
2,83732.0,92670.699919,69945.934107,3.0,1.0,87.0,2.0,2.0,2.0,4.0,...,0.191,0.000,0.000,0.007,0.001,0.0,0.000,0.000,0.000,0.000
3,83732.0,92670.699919,69945.934107,4.0,1.0,87.0,2.0,2.0,2.0,4.0,...,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000
4,83732.0,92670.699919,69945.934107,5.0,1.0,87.0,2.0,2.0,2.0,4.0,...,10.223,0.158,0.002,5.934,0.544,0.0,0.178,0.000,0.013,0.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100675,93702.0,67560.380806,55166.938286,13.0,1.0,87.0,2.0,2.0,4.0,3.0,...,1.224,0.019,0.001,1.027,0.144,0.0,0.014,0.011,0.004,0.082
100676,93702.0,67560.380806,55166.938286,15.0,1.0,87.0,2.0,2.0,4.0,3.0,...,0.001,0.000,0.000,0.007,0.018,0.0,0.000,0.000,0.000,0.000
100677,93702.0,67560.380806,55166.938286,16.0,1.0,87.0,2.0,2.0,4.0,3.0,...,0.037,0.000,0.000,0.098,0.004,0.0,0.000,0.000,0.000,0.000
100678,93702.0,67560.380806,55166.938286,17.0,1.0,87.0,2.0,2.0,4.0,3.0,...,9.111,0.025,0.000,1.682,0.126,0.0,0.000,0.000,0.000,0.000
